<center> <img src="profitroll.png">

# <center><span style="font-size: 50px; color: blue;">PROFITROLL DEMO</span></center>

<center><span style="font-size: 25px; color: purple;">This notebook shows how to use our package profitroll to perform a simulation</span></center>
<p></p>

<center><span style="font-size: 16px; color: black;">Please make sure you installed our module named <b><i>profitroll<i/></b> by running the following commands in a terminal:</span></center>
<p></p>

<center><code style="font-size: 16px; color: green;">$ cd PROFITROLL-Master/
$ python setup.py install

# Our module is named profitroll

In [ ]:
from profitroll.core.grid import Grid
from profitroll.core.state import State
from profitroll.core.simulation import Simulation
from profitroll.test.test_cases import v_stripe_test, bubble_test, gaussian_test

# Simulation parameters

In [ ]:
Lx = 2048e3
Ly = 1024e3
Nx = 256
Ny = 128


T = 5*3600 # Complete simulation is no more very long
dt = 300
Nt = int(T//dt)


dX = Nx//8  # used to shape the initial v-stripe data
dY = Ny//15

nb_state = 2  # number of instants in initial data

# We import our scientific methods

In [ ]:
from profitroll.methods.pseudo_spectral_wind import pseudo_spectral_wind
from profitroll.methods.wrap_advection_step_3P import wrap_advection_step_3P
from profitroll.methods.wrap_wv import wrap_wv
from profitroll.methods.end_pop import end_pop

Your methods have just been wrapped in such a way it fits our architechture, here is an example of a stupid method. Nothing of scientific, but you can see its signature, and basically understand how we expect a method to interact with data.

In [ ]:
def stupid_method(history, verbose, *args, **kwargs):
    last_state = history.state_list[-1]
    
    new_state = State.copy(last_state)
    new_state.t += dt
    
    print(new_state.t) if verbose else None
    
    new_state.vrs['theta_t'][int(new_state.t) % Nx, int(new_state.t) % Ny] = 1 # yes it is stupid
    
    history.append(new_state) # adding the new fresh state
    history.pop(0) # deleting the oldest state

# Simulation parameters and building

In [ ]:
methods = [pseudo_spectral_wind,
           wrap_advection_step_3P,wrap_wv,end_pop]

methods_kwargs = [{},
                  {'alpha_method' : 'damped_bicubic',
                   'order_alpha' : 2, 
                   'F_method' : 'damped_bicubic'},
                  {'alpha_method' : 'damped_bicubic',
                   'order_alpha' : 2, 
                   'F_method' : 'damped_bicubic'},
                  {}]

output_folder = 'output_test'
save_rate = 1
backup_rate = 10
verbose = 1 # displaying level, usefull to inspect what's going wrong

In [ ]:
# Creation of the test case
initialCDF = v_stripe_test('initial.nc', Lx, Ly, Nx, Ny, dt, nb_state, dX, dY)

In [ ]:
# Creation of the simulation object
mySim = Simulation(initialCDF,
                   methods, 
                   methods_kwargs,
                   output_folder,
                   verbose=verbose, 
                   name='testfb')

In [ ]:
# Run the simulation
mySim.run(T, Nt, save_rate, backup_rate, first_run=True) 

### if you want to extend the simulation a little bit...

We will extend the first run (5h of simulation) with a new one (2h). We can change the save rate. 

In [ ]:
T2 = 2*3600 
Nt2 = int(T2//dt)

In [ ]:
save_rate = 2
backup_rate = 6
mySim.run(T2, Nt2, save_rate, backup_rate, first_run=False)

# <center> Plot the results !

You will create an interactive object linked to a netCDF file (the result one). You can change the plotted variable, time and colormap. Don't worry it is re-opened and re-closed at each change you ask for so the file is not corrupted.

In [ ]:
from profitroll.display.interacting_plot import interactive_plot

In [ ]:
inter = interactive_plot('output_test/results_testfb.nc')
inter

In [ ]:
# Create a video
from filament.display import animate

In [ ]:
variable = 'Delta_z'

# the CDF file results.nc stored in the output_folder is used to 
# generate a video saved in the output_folder
animate.make_video(output_folder + '/results_testfb.nc', output_folder, variable)